# BERTopic

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import df_to_list

In [2]:
import pandas as pd
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, procesar_adela

caso = 'adela'
df = df_caso(caso)

df = procesar_adela(df)
df = df[df['opt_left'] == 'Producir el alimento contra déficit vitamínico']

df = process_df(df,'comment','sel',verbose=True)

train_indices = pd.read_csv('../utils/splits/train_indices_{}.csv'.format(caso), header=None)[0].tolist()
test_indices = pd.read_csv('../utils/splits/test_indices_{}.csv'.format(caso), header=None)[0].tolist()
print("Partición train-test cargada")

df_train = df.iloc[train_indices]
df_test = df.iloc[test_indices]

26 rows found with non string elements for column comment (0.40%)
Deleting 685 columns for which max target value is over 7 (10.58%)
5761 available rows after processing
Partición train-test cargada


Guardando la partición train-test

In [3]:
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

docs_train = df_to_list(df_train,'comment',tokenizador)
docs_test = df_to_list(df_test, 'comment',tokenizador)

## BERTopic algorithm

In [4]:
from IPython.display import Image, display
display(Image(url='https://maartengr.github.io/BERTopic/algorithm/modularity.svg'))

In [5]:
%%capture
from bertopic import BERTopic

In [6]:
model = BERTopic(language="spanish",min_topic_size=10,nr_topics=25,calculate_probabilities=True)

In [7]:
%%time
topics, probs = model.fit_transform(docs_train)

CPU times: user 1min 9s, sys: 3.22 s, total: 1min 12s
Wall time: 32.9 s


In [8]:
model.get_topic_freq().head(5)

,Topic,Count
6,0,1050
4,-1,967
2,1,912
5,2,785
0,3,450


In [9]:
model.get_topic(0)[:10]

[('vitamina', 0.04430047448982247),
 ('déficit', 0.036808130746463844),
 ('vitamínico', 0.03222842031962001),
 ('tradiciones', 0.025239648354502663),
 ('alimento', 0.02521085487689096),
 ('importante', 0.022403852312724775),
 ('producir', 0.020919201832737676),
 ('vitaminas', 0.019797244211233026),
 ('deficit', 0.01929089916901348),
 ('pueblo', 0.01907947053004304)]

Topicos para algunos documentos

In [18]:
for i, t in enumerate(model.topics_[:10]):
    print('Tópico {}:\ttexto: {}'.format(t,docs_train[i]))

Tópico 3:	texto: Adela debería priorizar salud personas alguna creencia grupo pequeño personas
Tópico 3:	texto: Dado Adela quiere utilizar fruto tradicional pueblo originario cuál esforzado cultivar mantener vigente debería respetar condiciones pueblo originario acepta uso fruto encargados hacerlo subsistir
Tópico 4:	texto: Si bien aún proyecto obligacion considerar opinión pueblos originarios demasía importante considerar opinión respeto cultura cosmovisión Lamenetablemente existe disgusto parte últimos proyecto existirá conflicto fututo hora producir producto Lo prudente realizar estudio baja escala implementando producción alimento compromiso incluir miembros comunidades indígenas llegar acuerdos conjunto
Tópico 1:	texto: Debe priorizar producción alimento pues trata deficiencia salud puede llegar generar problemas graves si tratan todas formas seguiría tratando lograr acuerdo pueblo originario si logra terminaría priorizando producción alimento
Tópico 5:	texto: Considero debería po

### Inferencia en conjunto test

In [17]:
string = 'Aun considero necesario resguardar mas tradiciones identitarias embargo producir alimento podría ayudar muchas personas entonces debería existir diálogo ambas partes llegar mejor solución'

model.transform([string])

([1],
 array([[0.00623311, 0.82206672, 0.00768691, 0.00166173, 0.01083431,
         0.02054779, 0.00183252, 0.01714246, 0.0252235 , 0.0014795 ,
         0.01401573, 0.01197215, 0.00298816, 0.00453383, 0.00547804,
         0.00146713, 0.00497382, 0.00583748, 0.00562884, 0.00699945,
         0.00382215, 0.00572577, 0.0060996 , 0.0057493 ]]))

In [26]:
output = model.transform(df_to_list(df_test.iloc[:3],'comment',tokenizador))

Imprimiendo los embeddings

In [22]:
output[1]

array([[2.60748306e-07, 9.99992559e-01, 3.21608300e-07, 6.95626648e-08,
        4.48658160e-07, 8.56978823e-07, 7.71827525e-08, 7.19084756e-07,
        1.05542780e-06, 6.18669934e-08, 5.84557413e-07, 5.01937386e-07,
        1.24967065e-07, 1.90642802e-07, 2.30500432e-07, 6.13418269e-08,
        2.08941013e-07, 2.44297437e-07, 2.35966077e-07, 2.93294476e-07,
        1.59180482e-07, 2.39199720e-07, 2.54531431e-07, 2.39504970e-07],
       [1.51270042e-02, 5.63852063e-01, 1.85522363e-02, 4.03375142e-03,
        2.60971278e-02, 5.03797106e-02, 4.48045340e-03, 4.19347811e-02,
        6.21433131e-02, 3.59700631e-03, 3.41155659e-02, 2.95563578e-02,
        7.26159209e-03, 1.12060388e-02, 1.35931254e-02, 3.56098996e-03,
        1.23000225e-02, 1.44383301e-02, 1.38568036e-02, 1.74728625e-02,
        9.33635005e-03, 1.40556912e-02, 1.49543592e-02, 1.40944613e-02],
       [2.05247640e-02, 4.04356184e-01, 2.55169290e-02, 5.50177737e-03,
        3.61538819e-02, 6.82212527e-02, 6.09088124e-03, 5.7739

Visualización

In [11]:
model.visualize_topics()

Podemos relacionar tópicos con palabras en específico

In [12]:
target = 'integridad'

topic_num = 5

similar_topics, similarity = model.find_topics(target, top_n=topic_num)
similar_topics

[13, 10, 17, 11, 14]

In [13]:
similarity

[0.5201327165496956,
 0.33273560867202556,
 0.32604916603811096,
 0.3214980358176935,
 0.32021616389699237]

In [14]:
data = [[] for _ in range(topic_num)]
max_w = 5

for i, t in enumerate(similar_topics):
    data[i].append(similarity[i])
    words_top = model.get_topics(t)
    data[i] += [tup[0] for tup in model.get_topic(t)[:max_w]]

In [15]:
from utils.exploracion import print_table_md

print("Tópicos relevantes para concepto: {}".format(target))

headers = ['Probabilidad'] + ['Palabra {}'.format(i+1) for i in range(max_w)]

print_table_md(headers, data)

Tópicos relevantes para concepto: integridad


| Probabilidad | Palabra 1 | Palabra 2 | Palabra 3 | Palabra 4 | Palabra 5 |
| --- | --- | --- | --- | --- | --- |
| 0.5201327165496956 | postura | mantengo | misma | no | mitad |
| 0.33273560867202556 | identitarias | tradiciones | identidad | resguardar | pueblos |
| 0.32604916603811096 | mayor | personas | pueblo | acuerdo | llegar |
| 0.3214980358176935 | proyecto | considerar | acuerdo | si | llevar |
| 0.32021616389699237 | ambas | partes | acuerdo | llegar | dos |


Guardando el modelo

In [16]:
model.save("modelos/BT-{}.bertopic".format(caso))

Para cargar:

`my_model = BERTopic.load("modelos/BT-caso")`